## Towards a final dataset

- Create final dataset from existing scripts

Environment

In [29]:
import pandas as pd
import dateutil
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re
import matplotlib as plt
import numpy as np

from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords

from collections import Counter

%matplotlib inline

In [45]:
pd.set_option('display.max_colwidth', -1)

Original data

In [4]:
data = pd.read_csv('20190313_TW_MigrantMigration.csv')
data[0:1]

,id_str,from_user,text,created_at,time,geo_coordinates,user_lang,in_reply_to_user_id_str,in_reply_to_screen_name,from_user_id_str,in_reply_to_status_id_str,source,profile_image_url,user_followers_count,user_friends_count,user_location,status_url,entities_str
0,1.102720e+18,darleneclarke5,"RT @ezralevant: Abdul Habash, a Syrian migrant...",Mon Mar 04 23:59:47 +0000 2019,04/03/2019 23:59:47,NaN,en,NaN,NaN,564257648.0,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",http://abs.twimg.com/sticky/default_profile_im...,197,579.0,NaN,http://twitter.com/darleneclarke5/statuses/110...,"{""hashtags"":[],""symbols"":[],""user_mentions"":[{..."


In [5]:
#New df
df=data

Current state of the dataframe

In [56]:
df[0:100]

,authorFollower,authorFriend,authorID,authorID_original,authorLang,authorLocation,authorPicture,authorScreen,authorScreen_original,tweetID,...,tweetURL,authorLang_new,authorPicture_new,tweetMethod_new,tweetDate,tweetDate_new,tweetSentiment_cat,authorLocation_new,isRetweet,tweetLang
0,197,579.0,5.642576e+08,NaN,en,NaN,http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png,darleneclarke5,NaN,0,...,http://twitter.com/darleneclarke5/statuses/1102720324875894785,English,Default,Web_Standard,Mon Mar 04 2019,2019-03-04,Very Negative,NaN,1,en
1,940,225.0,3.109106e+09,NaN,en,top your momma,http://pbs.twimg.com/profile_images/1062099415421411333/EZCmOCeu_normal.jpg,BlueelvisEric,NaN,1,...,http://twitter.com/BlueelvisEric/statuses/1102720294035353606,English,Personalized,Smartphone,Mon Mar 04 2019,2019-03-04,Negative,Not USA,1,en
2,103,275.0,1.083790e+18,NaN,en,NaN,http://pbs.twimg.com/profile_images/1083792878655045632/zQMarxaz_normal.jpg,Potus16Abe,NaN,2,...,http://twitter.com/Potus16Abe/statuses/1102720285478928384,English,Personalized,Web_Lite,Mon Mar 04 2019,2019-03-04,Positive,NaN,1,en
3,718,869.0,7.967720e+17,NaN,en,NaN,http://pbs.twimg.com/profile_images/1061628259761352707/45jSYb3T_normal.jpg,soozbo,NaN,3,...,http://twitter.com/soozbo/statuses/1102720268726951937,English,Personalized,Smartphone,Mon Mar 04 2019,2019-03-04,Negative,NaN,1,en
4,666,481.0,1.056307e+08,NaN,en,Netherlands,http://pbs.twimg.com/profile_images/378800000757810411/851d42a8b7e7efd00eded4d6efff5587_normal.jpeg,jp_dutch,NaN,4,...,http://twitter.com/jp_dutch/statuses/1102720262670290945,English,Personalized,Smartphone,Mon Mar 04 2019,2019-03-04,Very Negative,Not USA,1,en
5,125,232.0,1.078316e+09,NaN,en,NaN,http://pbs.twimg.com/profile_images/696140281092141057/8PzKc_uT_normal.jpg,RJ_Kendall,NaN,5,...,http://twitter.com/RJ_Kendall/statuses/1102720245624537088,English,Personalized,Web_Standard,Mon Mar 04 2019,2019-03-04,Very Negative,NaN,1,en
6,260,298.0,2.491415e+07,NaN,en,NaN,http://pbs.twimg.com/profile_images/862312515195162624/VZGvKgV5_normal.jpg,gabrieleverett,NaN,6,...,http://twitter.com/gabrieleverett/statuses/1102720236338393094,English,Personalized,Smartphone,Mon Mar 04 2019,2019-03-04,Positive,NaN,1,en
7,334,521.0,7.150480e+17,NaN,ja,F研 なのるな Attellui,http://pbs.twimg.com/profile_images/1096403448205959173/dAcHrHQa_normal.jpg,lT1g0n8T2,NaN,7,...,http://twitter.com/lT1g0n8T2/statuses/1102720208559435776,Other,Personalized,Smartphone,Mon Mar 04 2019,2019-03-04,Neutral,Not USA,1,en
8,240,623.0,8.066630e+17,NaN,fi,NaN,http://pbs.twimg.com/profile_images/975807858343399425/iQByS93x_normal.jpg,suse_______,NaN,8,...,http://twitter.com/suse_______/statuses/1102720201135673347,Other,Personalized,Smartphone,Mon Mar 04 2019,2019-03-04,Positive,NaN,1,en
9,2754,2551.0,9.819028e+07,NaN,de,Bleckede,http://pbs.twimg.com/profile_images/981563020508827648/KUxixgWS_normal.jpg,ffd365,NaN,9,...,http://twitter.com/ffd365/statuses/1102720172845076481,Other,Personalized,Other,Mon Mar 04 2019,2019-03-04,Negative,Not USA,0,de


Unique ID variable for each tweet

In [7]:
df.reset_index(inplace=True)
df.rename(columns={'index':'tweetID'}, inplace=True)

Sentiment Score

In [8]:
#create an object called map_ID_text
#containing a dictionary
#with a tuple for each row in the df where
#element 1 of the tuple is the value in tweet ID
#and element 2 of the tuple is the value in text

map_ID_text = dict((tuple(x) for x in df[['tweetID', 'text']].values))

In [9]:
#create an object called analyzer
#containing the SentimentIntensityAnalyzer() function
#create an object called map_ID_valence_dict
#containing an empty dictionary
#loop through items in map_ID_text
#calculate sentiment score (that is, valence) of each text
#append score to map_ID_valence_dict, indexed by ID

analyzer = SentimentIntensityAnalyzer()

map_ID_valence_dict = {}

for ID, text in map_ID_text.items():
    map_ID_valence_dict[ID] = analyzer.polarity_scores(text)

In [10]:
#create an object called map_ID_senti_score
#containing an empty dictionary
#loop through items in map_ID_valence_dict
#containing an empty dictionary
#for pair in map_ID_text
#append "compound" element of valence to map_ID_valence_dict, indexed by ID

map_ID_senti_score = {} # Mapping IDs to a single score 'combined'

for ID, val in map_ID_valence_dict.items():
    map_ID_senti_score[ID] = val['compound']

In [11]:
#create new column for sentiment score
#fill with items from map_ID_senti_score
df['SentiScore_VADER'] = df['tweetID'].map(map_ID_senti_score)

Drop unnecessary columns

In [12]:
#Drop the "text" and "entities" columns that Sunyam is working on. 
#Also dropping time because it is redundant with "created_at"
df = df.drop(['id_str','time','entities_str', 'geo_coordinates'], axis=1)

In [13]:
#Rename columns
df = df.rename(columns={"id_str":"tweetID",
                        "from_user":"authorScreen",
                        "created_at":"tweetTimeDate",
                        "geo_coordinates":"tweetGeo",
                        "user_lang":"authorLang",
                        "in_reply_to_user_id_str":"authorID_original",
                        "in_reply_to_screen_name":"authorScreen_original",
                        "from_user_id_str":"authorID",
                        "in_reply_to_status_id_str":"tweetID_original",
                        "source":"tweetMethod", 
                        "profile_image_url":"authorPicture",
                        "user_followers_count":"authorFollower",
                        "user_friends_count":"authorFriend",
                        "user_location":"authorLocation",
                        "status_url":"tweetURL",
                        "text":"tweetText",
                        "SentiScore_VADER": "tweetSentiment"})

In [14]:
#Group author and tweet variables together in the dataset.
df = df.sort_index(axis=1)

Recode variables

In [15]:
df["authorLang_new"] = df["authorLang"].astype('category')
df['authorLang_new'] = df['authorLang_new'].replace(regex=r'en.*', value='English')
df['authorLang_new'] = df['authorLang_new'].replace(regex=r'^((?!English)\S*)', value='Other')
df['authorLang_new'].value_counts()

English    109347
Other       14177
Name: authorLang_new, dtype: int64

In [16]:
df["authorPicture_new"] = df["authorPicture"].astype('category')
df['authorPicture_new'] = df['authorPicture_new'].replace(regex=r'^.*default_profile_images\S*', value='Default')
df['authorPicture_new'] = df['authorPicture_new'].replace(regex=r'^.*/profile_images\S*', value='Personalized')
df['authorPicture_new'].value_counts()

Personalized    117131
Default           6393
Name: authorPicture_new, dtype: int64

In [17]:
df["tweetMethod_new"] = df["tweetMethod"].astype('category')
df['tweetMethod_new'] = df['tweetMethod_new'].replace(regex=r'^.*Twitter for iPhone\S*', value='Smartphone')
df['tweetMethod_new'] = df['tweetMethod_new'].replace(regex=r'^.*Twitter for Android\S*', value='Smartphone')
df['tweetMethod_new'] = df['tweetMethod_new'].replace(regex=r'^.*Twitter Web Client\S*', value='Web_Standard')
df['tweetMethod_new'] = df['tweetMethod_new'].replace(regex=r'^.*Twitter Web App\S*', value='Web_Lite')
df['tweetMethod_new'] = df['tweetMethod_new'].replace(regex=r'^.*Twitter for iPad\S*', value='Web_Lite')
df['tweetMethod_new'].value_counts()[0:10]

Smartphone                                                                             74524
Web_Standard                                                                           24144
Web_Lite                                                                               16764
<a href="http://www.facebook.com/twitter" rel="nofollow">Facebook</a>                   1300
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>     1079
<a href="https://ifttt.com" rel="nofollow">IFTTT</a>                                     500
<a href="https://dlvrit.com/" rel="nofollow">dlvr.it</a>                                 474
<a href="https://www.hootsuite.com" rel="nofollow">Hootsuite Inc.</a>                    457
<a href="https://buffer.com" rel="nofollow">Buffer</a>                                   360
<a href="http://publicize.wp.com/" rel="nofollow">WordPress.com</a>                      266
Name: tweetMethod_new, dtype: int64

In [18]:
df['tweetMethod_new'] = df['tweetMethod_new'].replace(regex=r'^.*href.*', value='Other')
df['tweetMethod_new'] = df['tweetMethod_new'].replace(regex=r'^Web_Standard.*(T1).*', value='Other')
df['tweetMethod_new'].value_counts()

Smartphone      74524
Web_Standard    24144
Web_Lite        16764
Other            8092
Name: tweetMethod_new, dtype: int64

In [19]:
df['tweetDate'] = df['tweetTimeDate'].replace(regex=r'\s\d{2}:.*:\d{2}\s.{5}', value='')
df['tweetDate_new'] = df['tweetDate'].apply(dateutil.parser.parse, dayfirst=True)
df['tweetDate_new'].value_counts()[0:10]

2019-02-28    2913
2019-01-26    2910
2019-02-22    2909
2019-02-10    2906
2019-02-04    2905
2019-02-08    2902
2019-02-11    2901
2019-01-28    2900
2019-03-01    2897
2019-02-23    2896
Name: tweetDate_new, dtype: int64

In [20]:
#Recode the numerical sentiment scores into categorical data
bins = [-1, -0.8, -0.05, 0.05, 0.8, 1]
names = ['Very Negative', 'Negative', 'Neutral', 'Positive', 'Very Positive']
df['tweetSentiment_cat'] = pd.cut(df['tweetSentiment'], bins, labels=names)
df['tweetSentiment_cat'].value_counts()

Negative         46370
Neutral          27895
Positive         27009
Very Negative    16475
Very Positive     5775
Name: tweetSentiment_cat, dtype: int64

Author locations

In [21]:
df['authorLocation_new'] = df['authorLocation']
df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'.*USA.*', value='United States')
df['authorLocation_new'].value_counts()[0:9]

United States              15625
Washington, DC               680
London                       652
London, England              648
Canada                       561
United Kingdom               530
England, United Kingdom      516
Los Angeles, CA              476
New York, NY                 475
Name: authorLocation_new, dtype: int64

In [22]:
USA = pd.read_csv('https://raw.githubusercontent.com/cphalpert/census-regions/master/us%20census%20bureau%20regions%20and%20divisions.csv')
USA[0:10]

,State,State Code,Region,Division
0,Alaska,AK,West,Pacific
1,Alabama,AL,South,East South Central
2,Arkansas,AR,South,West South Central
3,Arizona,AZ,West,Mountain
4,California,CA,West,Pacific
5,Colorado,CO,West,Mountain
6,Connecticut,CT,Northeast,New England
7,District of Columbia,DC,South,South Atlantic
8,Delaware,DE,South,South Atlantic
9,Florida,FL,South,South Atlantic


In [23]:
states = USA['State'].tolist()
print(states)

['Alaska', 'Alabama', 'Arkansas', 'Arizona', 'California', 'Colorado', 'Connecticut', 'District of Columbia', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Iowa', 'Idaho', 'Illinois', 'Indiana', 'Kansas', 'Kentucky', 'Louisiana', 'Massachusetts', 'Maryland', 'Maine', 'Michigan', 'Minnesota', 'Missouri', 'Mississippi', 'Montana', 'North Carolina', 'North Dakota', 'Nebraska', 'New Hampshire', 'New Jersey', 'New Mexico', 'Nevada', 'New York', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Virginia', 'Vermont', 'Washington', 'Wisconsin', 'West Virginia', 'Wyoming']


In [24]:
for x in states:
    df['authorLocation_new'] = df['authorLocation_new'].replace(regex='.*' + str(x) + '.*', value='United States')
df['authorLocation_new'][df['authorLocation_new']!="United States"].value_counts()[0:9]

London                     652
London, England            648
Canada                     561
United Kingdom             530
England, United Kingdom    516
Los Angeles, CA            476
Australia                  391
UK                         375
France                     339
Name: authorLocation_new, dtype: int64

In [25]:
stateCodes = USA['State Code'].tolist()
print(stateCodes)

['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']


In [26]:
for x in stateCodes:
    df['authorLocation_new'] = df['authorLocation_new'].replace(regex='^.*' + str(x) + '.*$', value='United States')
df['authorLocation_new'][df['authorLocation_new']!="United States"].value_counts()[0:9]

London                     652
London, England            648
Canada                     561
United Kingdom             530
England, United Kingdom    516
Australia                  391
UK                         375
France                     339
Earth                      298
Name: authorLocation_new, dtype: int64

In [27]:
df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^America.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Chicago.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^NYC.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Los Angeles.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^San Francisco.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Boston.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^United States of America.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^NC$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^NY$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^US$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^NJ$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^CA$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^TEXAS$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^U.S.A.$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Seattle$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Atlanta$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^San Diego$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Philadelphia$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Midwest$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Pacific Northwest$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Brooklyn$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^florida$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Las Vegas$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^usa.*$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Bay Area$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^texas$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^San Jose$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Miami$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^So Cal$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Tucson$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^The South$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Dallas$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^nyc$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^usa$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^los angeles$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Houston$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Tx$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Upstate NY$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Usa$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Silicon Valley$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^U.S.$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^U.S.A.$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Troy, MICHIGAN$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^THE SOUTH$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^U.S.A$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Minneapolis', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Western PA', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^SF Bay Area', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Columbus', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^chicago$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Conway Ar$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Denver$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Nashville$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Puget Sound$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^new york$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Pittsburgh$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Commiefornia$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'.*\bUS\b.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'.*Rocky Mountains.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'.*Manhattan.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'.*Vero Beach.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'.*california.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Ca$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^ohio$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'.*United States.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'.*Western NY.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'.*Southern CA.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'.*NEW YORK.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'.*United States of America.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'.*Trump Nation.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^NY.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'United Sates of America', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'From Sea to Shining Sea', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'Portland OR', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^ca$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'.*United States\..*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'.*Merica.*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'.*The Dirty South\, U\.S\.A\..*', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Cali$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^In.*Ca\.$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Buffalo NY$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Philly$', value='United States')
df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^wisconsin$', value='United States')
df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^New Orleans$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^Long Island$', value='United States')
df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^los angeles, ca$', value='United States')

df['authorLocation_new'] = df['authorLocation_new'].replace(regex=r'^.*Nashville.*', value='United States')

In [28]:
filter1 = df['authorLocation_new']=="United States"
filter2 = df['authorLocation_new'].isnull()
df['authorLocation_new'].where(filter1 | filter2, "Not USA", inplace = True) 
df['authorLocation_new'].value_counts()

Not USA          42788
United States    42418
Name: authorLocation_new, dtype: int64

## Adding a column to the Dataframe for Retweets (1=RT | 0=otherwise):

In [30]:
map_tweetID_RTorNot = {}

map_tweetID_tweetText = dict([tuple(x) for x in df[['tweetID', 'tweetText']].values])

for ID, text in map_tweetID_tweetText.items():
    if text.startswith('RT'):
        map_tweetID_RTorNot[ID] = 1
    else:
        map_tweetID_RTorNot[ID] = 0

In [31]:
# Reconfirming the values:
from collections import Counter
Counter(map_tweetID_RTorNot.values())

Counter({1: 97185, 0: 26339})

In [32]:
df['isRetweet'] = df['tweetID'].map(map_tweetID_RTorNot)

In [33]:
df.head(10)

,authorFollower,authorFriend,authorID,authorID_original,authorLang,authorLocation,authorPicture,authorScreen,authorScreen_original,tweetID,...,tweetTimeDate,tweetURL,authorLang_new,authorPicture_new,tweetMethod_new,tweetDate,tweetDate_new,tweetSentiment_cat,authorLocation_new,isRetweet
0,197,579.0,5.642576e+08,NaN,en,NaN,http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png,darleneclarke5,NaN,0,...,Mon Mar 04 23:59:47 +0000 2019,http://twitter.com/darleneclarke5/statuses/1102720324875894785,English,Default,Web_Standard,Mon Mar 04 2019,2019-03-04,Very Negative,NaN,1
1,940,225.0,3.109106e+09,NaN,en,top your momma,http://pbs.twimg.com/profile_images/1062099415421411333/EZCmOCeu_normal.jpg,BlueelvisEric,NaN,1,...,Mon Mar 04 23:59:40 +0000 2019,http://twitter.com/BlueelvisEric/statuses/1102720294035353606,English,Personalized,Smartphone,Mon Mar 04 2019,2019-03-04,Negative,Not USA,1
2,103,275.0,1.083790e+18,NaN,en,NaN,http://pbs.twimg.com/profile_images/1083792878655045632/zQMarxaz_normal.jpg,Potus16Abe,NaN,2,...,Mon Mar 04 23:59:38 +0000 2019,http://twitter.com/Potus16Abe/statuses/1102720285478928384,English,Personalized,Web_Lite,Mon Mar 04 2019,2019-03-04,Positive,NaN,1
3,718,869.0,7.967720e+17,NaN,en,NaN,http://pbs.twimg.com/profile_images/1061628259761352707/45jSYb3T_normal.jpg,soozbo,NaN,3,...,Mon Mar 04 23:59:34 +0000 2019,http://twitter.com/soozbo/statuses/1102720268726951937,English,Personalized,Smartphone,Mon Mar 04 2019,2019-03-04,Negative,NaN,1
4,666,481.0,1.056307e+08,NaN,en,Netherlands,http://pbs.twimg.com/profile_images/378800000757810411/851d42a8b7e7efd00eded4d6efff5587_normal.jpeg,jp_dutch,NaN,4,...,Mon Mar 04 23:59:32 +0000 2019,http://twitter.com/jp_dutch/statuses/1102720262670290945,English,Personalized,Smartphone,Mon Mar 04 2019,2019-03-04,Very Negative,Not USA,1
5,125,232.0,1.078316e+09,NaN,en,NaN,http://pbs.twimg.com/profile_images/696140281092141057/8PzKc_uT_normal.jpg,RJ_Kendall,NaN,5,...,Mon Mar 04 23:59:28 +0000 2019,http://twitter.com/RJ_Kendall/statuses/1102720245624537088,English,Personalized,Web_Standard,Mon Mar 04 2019,2019-03-04,Very Negative,NaN,1
6,260,298.0,2.491415e+07,NaN,en,NaN,http://pbs.twimg.com/profile_images/862312515195162624/VZGvKgV5_normal.jpg,gabrieleverett,NaN,6,...,Mon Mar 04 23:59:26 +0000 2019,http://twitter.com/gabrieleverett/statuses/1102720236338393094,English,Personalized,Smartphone,Mon Mar 04 2019,2019-03-04,Positive,NaN,1
7,334,521.0,7.150480e+17,NaN,ja,F研 なのるな Attellui,http://pbs.twimg.com/profile_images/1096403448205959173/dAcHrHQa_normal.jpg,lT1g0n8T2,NaN,7,...,Mon Mar 04 23:59:19 +0000 2019,http://twitter.com/lT1g0n8T2/statuses/1102720208559435776,Other,Personalized,Smartphone,Mon Mar 04 2019,2019-03-04,Neutral,Not USA,1
8,240,623.0,8.066630e+17,NaN,fi,NaN,http://pbs.twimg.com/profile_images/975807858343399425/iQByS93x_normal.jpg,suse_______,NaN,8,...,Mon Mar 04 23:59:18 +0000 2019,http://twitter.com/suse_______/statuses/1102720201135673347,Other,Personalized,Smartphone,Mon Mar 04 2019,2019-03-04,Positive,NaN,1
9,2754,2551.0,9.819028e+07,NaN,de,Bleckede,http://pbs.twimg.com/profile_images/981563020508827648/KUxixgWS_normal.jpg,ffd365,NaN,9,...,Mon Mar 04 23:59:11 +0000 2019,http://twitter.com/ffd365/statuses/1102720172845076481,Other,Personalized,Other,Mon Mar 04 2019,2019-03-04,Negative,Not USA,0


## Adding a column to the Dataframe for tweet language:

In [34]:
# Since authorLang is our default:
tweetID_tweetText_authorLang = [tuple(x) for x in df[['tweetID', 'tweetText', 'authorLang']].values]
tweetID_tweetText_authorLang[:3] # preview

[(0,
  'RT @ezralevant: Abdul Habash, a Syrian migrant to Canada, made a terrifying bomb threat against Jason Kenney. He and his dad told our reporter it’s no big deal — in fact, they’re mad the cops won’t give him his computer and phone back. #terrorists https://t.co/AujC0eU4Sk',
  'en'),
 (1,
  'RT @1776Stonewall: https://t.co/CyucESMlFM  Texas health officials report that nearly 200 people contracted mumps in migrant detention facilities located across the state so far this fiscal year.',
  'en'),
 (2,
  "RT @EmeraldRobinson: Trump's base was promised a border wall, and the end of chain migration and birthright citizenship. People inside the administration who think they can pass amnesty bills and still win 2020 are playing with fire. https://t.co/36Fc7dxpNY",
  'en')]

In [36]:
import langdetect
# To ensure consistent results (cuz it's non-deterministic)
from langdetect import DetectorFactory
DetectorFactory.seed = 0

In [37]:
map_tweetID_tweetLang = {}

failed_examples = [] # keep track of examples where langdetect failed

for tup in tweetID_tweetText_authorLang:
    ID = tup[0]
    text = tup[1]
    author_lang = tup[2]
    
    try:
        tweet_lang = langdetect.detect(text)
    except: # default to authorLang
#         tweet_lang = author_lang
        tweet_lang = "BLAH"
        failed_examples.append(text)
    
    if '-' in author_lang: # Resolving cases such as 'en-gb' to be same as 'en'
        if tweet_lang in author_lang: # check if langdetect() got it right
            tweet_lang = author_lang
            
    map_tweetID_tweetLang[ID] = tweet_lang

In [38]:
failed_examples[:5] # preview cases where langdetect didn't work

['https://t.co/yjpKHDo6Fc',
 'https://t.co/K9rFn4rL6r',
 'https://t.co/VmQvRgpOZV',
 'https://t.co/UW4GYhFHLX',
 'https://t.co/4P7T6DSy96']

In [39]:
df['tweetLang'] = df['tweetID'].map(map_tweetID_tweetLang)
df.head()

,authorFollower,authorFriend,authorID,authorID_original,authorLang,authorLocation,authorPicture,authorScreen,authorScreen_original,tweetID,...,tweetURL,authorLang_new,authorPicture_new,tweetMethod_new,tweetDate,tweetDate_new,tweetSentiment_cat,authorLocation_new,isRetweet,tweetLang
0,197,579.0,5.642576e+08,NaN,en,NaN,http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png,darleneclarke5,NaN,0,...,http://twitter.com/darleneclarke5/statuses/1102720324875894785,English,Default,Web_Standard,Mon Mar 04 2019,2019-03-04,Very Negative,NaN,1,en
1,940,225.0,3.109106e+09,NaN,en,top your momma,http://pbs.twimg.com/profile_images/1062099415421411333/EZCmOCeu_normal.jpg,BlueelvisEric,NaN,1,...,http://twitter.com/BlueelvisEric/statuses/1102720294035353606,English,Personalized,Smartphone,Mon Mar 04 2019,2019-03-04,Negative,Not USA,1,en
2,103,275.0,1.083790e+18,NaN,en,NaN,http://pbs.twimg.com/profile_images/1083792878655045632/zQMarxaz_normal.jpg,Potus16Abe,NaN,2,...,http://twitter.com/Potus16Abe/statuses/1102720285478928384,English,Personalized,Web_Lite,Mon Mar 04 2019,2019-03-04,Positive,NaN,1,en
3,718,869.0,7.967720e+17,NaN,en,NaN,http://pbs.twimg.com/profile_images/1061628259761352707/45jSYb3T_normal.jpg,soozbo,NaN,3,...,http://twitter.com/soozbo/statuses/1102720268726951937,English,Personalized,Smartphone,Mon Mar 04 2019,2019-03-04,Negative,NaN,1,en
4,666,481.0,1.056307e+08,NaN,en,Netherlands,http://pbs.twimg.com/profile_images/378800000757810411/851d42a8b7e7efd00eded4d6efff5587_normal.jpeg,jp_dutch,NaN,4,...,http://twitter.com/jp_dutch/statuses/1102720262670290945,English,Personalized,Smartphone,Mon Mar 04 2019,2019-03-04,Very Negative,Not USA,1,en


In [40]:
df.to_csv('../20190403_AM_Data.csv',index=False)

In [43]:
df[39029:39040]

,authorFollower,authorFriend,authorID,authorID_original,authorLang,authorLocation,authorPicture,authorScreen,authorScreen_original,tweetID,...,tweetURL,authorLang_new,authorPicture_new,tweetMethod_new,tweetDate,tweetDate_new,tweetSentiment_cat,authorLocation_new,isRetweet,tweetLang
39029,82,362.0,1.025550e+18,NaN,en,NaN,http://pbs.twimg.com/profile_images/1045503310009065472/xCaDL6S__normal.jpg,CTagori,NaN,39029,...,http://twitter.com/CTagori/statuses/1097993163271225345,English,Personalized,Smartphone,Tue Feb 19 2019,2019-02-19,Negative,NaN,1,en
39030,1148,4464.0,7.087810e+17,7.836390e+17,en-GB,NaN,http://pbs.twimg.com/profile_images/708781154481152000/dATs-xGX_normal.jpg,Brexitbabe,Remoaniac,39030,...,http://twitter.com/Brexitbabe/statuses/1097993162726010883,English,Personalized,Web_Lite,Tue Feb 19 2019,2019-02-19,Neutral,NaN,0,en-GB
39031,1878,1694.0,3.163235e+08,NaN,en,Mesa\r,http://pbs.twimg.com/profile_images/575027765763698688/Dzjr_xki_normal.jpeg,PIatoQuotes,NaN,39031,...,http://twitter.com/PIatoQuotes/statuses/1097993161446772736,English,Personalized,Other,Tue Feb 19 2019,2019-02-19,Negative,Not USA,0,en
39032,1148,4464.0,7.087810e+17,7.836390e+17,en-GB,NaN,http://pbs.twimg.com/profile_images/708781154481152000/dATs-xGX_normal.jpg,Brexitbabe,Remoaniac,39032,...,http://twitter.com/Brexitbabe/statuses/1097993135840477185,English,Personalized,Web_Lite,Tue Feb 19 2019,2019-02-19,Neutral,NaN,0,en-GB
39033,88,1502.0,8.113740e+17,NaN,en,"Maine, USA",http://pbs.twimg.com/profile_images/824654789510307840/M3c3Ksiv_normal.jpg,k_Bradstreet,NaN,39033,...,http://twitter.com/k_Bradstreet/statuses/1097993132875091968,English,Personalized,Smartphone,Tue Feb 19 2019,2019-02-19,Positive,United States,1,en
39034,341,1515.0,3.303450e+08,NaN,en,Tampa,http://pbs.twimg.com/profile_images/1429080971/Pam_normal.jpg,PamManos,NaN,39034,...,http://twitter.com/PamManos/statuses/1097993131155443712,English,Personalized,Smartphone,Tue Feb 19 2019,2019-02-19,Very Negative,Not USA,1,en
39035,2567,1901.0,2.473049e+09,NaN,en,"California, USA",http://pbs.twimg.com/profile_images/760071858599305216/na2C8His_normal.jpg,DiSwanson77,NaN,39035,...,http://twitter.com/DiSwanson77/statuses/1097993131042041856,English,Personalized,Smartphone,Tue Feb 19 2019,2019-02-19,Very Negative,United States,1,en
39036,434,1488.0,1.297083e+08,NaN,en,Los Angeles ca,http://pbs.twimg.com/profile_images/963665559966625795/d4F3d9le_normal.jpg,lorena3511,NaN,39036,...,http://twitter.com/lorena3511/statuses/1097993126289977344,English,Personalized,Smartphone,Tue Feb 19 2019,2019-02-19,Positive,United States,1,en
39037,199,2244.0,2.396478e+09,NaN,en,NaN,http://pbs.twimg.com/profile_images/446256407386419201/vbqiWJ8M_normal.jpeg,JohnAlmoni,NaN,39037,...,http://twitter.com/JohnAlmoni/statuses/1097993126189428736,English,Personalized,Smartphone,Tue Feb 19 2019,2019-02-19,Neutral,NaN,1,en
39038,42406,47485.0,2.972957e+09,NaN,en,United States,http://pbs.twimg.com/profile_images/764127710759137280/PTXXPjI5_normal.jpg,zingars_,NaN,39038,...,http://twitter.com/zingars_/statuses/1097993118841012224,English,Personalized,Smartphone,Tue Feb 19 2019,2019-02-19,Positive,United States,1,en


In [47]:
df['authorLocation'] = df['authorLocation'].replace(regex=r'\r', value='')

In [48]:
df[39029:39040]

,authorFollower,authorFriend,authorID,authorID_original,authorLang,authorLocation,authorPicture,authorScreen,authorScreen_original,tweetID,...,tweetURL,authorLang_new,authorPicture_new,tweetMethod_new,tweetDate,tweetDate_new,tweetSentiment_cat,authorLocation_new,isRetweet,tweetLang
39029,82,362.0,1.025550e+18,NaN,en,NaN,http://pbs.twimg.com/profile_images/1045503310009065472/xCaDL6S__normal.jpg,CTagori,NaN,39029,...,http://twitter.com/CTagori/statuses/1097993163271225345,English,Personalized,Smartphone,Tue Feb 19 2019,2019-02-19,Negative,NaN,1,en
39030,1148,4464.0,7.087810e+17,7.836390e+17,en-GB,NaN,http://pbs.twimg.com/profile_images/708781154481152000/dATs-xGX_normal.jpg,Brexitbabe,Remoaniac,39030,...,http://twitter.com/Brexitbabe/statuses/1097993162726010883,English,Personalized,Web_Lite,Tue Feb 19 2019,2019-02-19,Neutral,NaN,0,en-GB
39031,1878,1694.0,3.163235e+08,NaN,en,Mesa,http://pbs.twimg.com/profile_images/575027765763698688/Dzjr_xki_normal.jpeg,PIatoQuotes,NaN,39031,...,http://twitter.com/PIatoQuotes/statuses/1097993161446772736,English,Personalized,Other,Tue Feb 19 2019,2019-02-19,Negative,Not USA,0,en
39032,1148,4464.0,7.087810e+17,7.836390e+17,en-GB,NaN,http://pbs.twimg.com/profile_images/708781154481152000/dATs-xGX_normal.jpg,Brexitbabe,Remoaniac,39032,...,http://twitter.com/Brexitbabe/statuses/1097993135840477185,English,Personalized,Web_Lite,Tue Feb 19 2019,2019-02-19,Neutral,NaN,0,en-GB
39033,88,1502.0,8.113740e+17,NaN,en,"Maine, USA",http://pbs.twimg.com/profile_images/824654789510307840/M3c3Ksiv_normal.jpg,k_Bradstreet,NaN,39033,...,http://twitter.com/k_Bradstreet/statuses/1097993132875091968,English,Personalized,Smartphone,Tue Feb 19 2019,2019-02-19,Positive,United States,1,en
39034,341,1515.0,3.303450e+08,NaN,en,Tampa,http://pbs.twimg.com/profile_images/1429080971/Pam_normal.jpg,PamManos,NaN,39034,...,http://twitter.com/PamManos/statuses/1097993131155443712,English,Personalized,Smartphone,Tue Feb 19 2019,2019-02-19,Very Negative,Not USA,1,en
39035,2567,1901.0,2.473049e+09,NaN,en,"California, USA",http://pbs.twimg.com/profile_images/760071858599305216/na2C8His_normal.jpg,DiSwanson77,NaN,39035,...,http://twitter.com/DiSwanson77/statuses/1097993131042041856,English,Personalized,Smartphone,Tue Feb 19 2019,2019-02-19,Very Negative,United States,1,en
39036,434,1488.0,1.297083e+08,NaN,en,Los Angeles ca,http://pbs.twimg.com/profile_images/963665559966625795/d4F3d9le_normal.jpg,lorena3511,NaN,39036,...,http://twitter.com/lorena3511/statuses/1097993126289977344,English,Personalized,Smartphone,Tue Feb 19 2019,2019-02-19,Positive,United States,1,en
39037,199,2244.0,2.396478e+09,NaN,en,NaN,http://pbs.twimg.com/profile_images/446256407386419201/vbqiWJ8M_normal.jpeg,JohnAlmoni,NaN,39037,...,http://twitter.com/JohnAlmoni/statuses/1097993126189428736,English,Personalized,Smartphone,Tue Feb 19 2019,2019-02-19,Neutral,NaN,1,en
39038,42406,47485.0,2.972957e+09,NaN,en,United States,http://pbs.twimg.com/profile_images/764127710759137280/PTXXPjI5_normal.jpg,zingars_,NaN,39038,...,http://twitter.com/zingars_/statuses/1097993118841012224,English,Personalized,Smartphone,Tue Feb 19 2019,2019-02-19,Positive,United States,1,en


In [53]:
df['tweetText'].str.contains(r'^\r').value_counts()

False    123524
Name: tweetText, dtype: int64

In [55]:
df.to_csv("20190406_AM_Data.csv", index=False)